In [ ]:
# !pip install ImageHash

In [10]:
from pymongo import MongoClient
import os
from pathlib import Path
from PIL import Image
import imagehash
from tqdm.notebook import tqdm

from lib.parallel import parallel

In [11]:
from lib.fix_nested_tqdm import fix_nested_tqdm

fix_nested_tqdm()

In [4]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
imagesfiles = db['imagefiles']

In [5]:
images_dir = Path('./images')

In [6]:
class ImageFile ():
    _attrs = [
        'id',
        'post_id',
        'image_id',
        'short_image_id',
        'album',
        'index_in_album',
        'image_type',
        'file_path',
        'filename',
        'basename',
        'ext',
        'source_platform',
        'source',
        'size',
        'width',
        'height',
        'ahash',
        'phash',
        'pshash',
        'dhash',
        'whash'
    ]

    def __init__ (self, file_path):
        self.file_path = file_path
        self.filename = os.path.basename(file_path)
        self.basename, self.ext = os.path.splitext(self.filename)
        self._im = Image.open(file_path)
        self.width, self.height = self._im.size

        self._file_path_tokens = self.file_path.split('/')

    def digest (self):
        return {a:getattr(self, a) for a in ImageFile._attrs}

    @property
    def size (self):
        return os.path.getsize(self.file_path)

    @property
    def image_type (self):
        return self._file_path_tokens[1]

    @property
    def source_platform (self):
        return self._file_path_tokens[2]

    @property
    def source (self):
        return self._file_path_tokens[3]

    @property
    def album (self):
        return self._file_path_tokens[4] if len(self._file_path_tokens) > 5 else ''

    @property
    def index_in_album (self):
        return int(self.basename.split('-')[0]) if '-' in self.basename else 0

    @property
    def id (self):
        return self.album if self.album else self.basename

    @property
    def post_id (self):
        return f"{self.source_platform}/{self.source}/{self.id}"

    @property
    def image_id (self):
        return f"{self.post_id}:{self.index_in_album}"

    @property
    def short_image_id (self):
        return f"{self.id}:{self.index_in_album}"

    @property
    def ahash (self):
        if not hasattr(self, '_ahash'):
            self._ahash = imagehash.average_hash(self._im)
        return str(self._ahash)

    @property
    def phash (self):
        if not hasattr(self, '_phash'):
            self._phash = imagehash.phash(self._im)
        return str(self._phash)

    @property
    def pshash (self):
        if not hasattr(self, '_pshash'):
            self._pshash = imagehash.phash_simple(self._im)
        return str(self._pshash)

    @property
    def dhash (self):
        if not hasattr(self, '_dhash'):
            self._dhash = imagehash.dhash(self._im)
        return str(self._dhash)

    @property
    def whash (self):
        if not hasattr(self, '_whash'):
            self._whash = imagehash.whash(self._im)
        return str(self._whash)

In [7]:
def digest_file (name, root=''):
    basename, ext = os.path.splitext(name)
    imageFile = ImageFile(os.path.join(root, name))
    if not imagesfiles.find_one({'file_path': imageFile.file_path}, {'_id': 1}):
        imagesfiles.insert_one(imageFile.digest())

with tqdm(os.walk(images_dir)) as t:
    for root, dirs, files in t:
        t.set_postfix(root=root)
        parallel(digest_file, files, params_dict={'root': root}, tqdm_postfix=root, leave=None)

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:2766: DecompressionBombWarning: Image size (135364608 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:2766: DecompressionBombWarning: Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [8]:
# import warnings
# warnings.filterwarnings("error")
# %%capture cap_out --no-stderr
# for root, dirs, files in os.walk(images_dir):
#     for name in files:
#         try:
#             imageFile = ImageFile(os.path.join(root, name))
#             imageFile.digest()
#         except Exception as inst:
#             print(f"Error digesting image: {os.path.join(root, name)} {inst}")

In [9]:
# with open('error.txt', 'w') as f:
#     f.write(cap_out.stdout)